## GPT 3.5 16k trial

In [ ]:
from openai import OpenAI
client = OpenAI(api_key = '<insert API key here>' )        # key removed for privacy, use appropriate key here
import time

In [ ]:
import os
import glob
import pandas as pd

directory_path = 'Patients_Datewise_Texts_With_First_Occurance'

csv_files = glob.glob(os.path.join(directory_path, '*.csv'))
dfs = []
for csv_file in csv_files:
    df = pd.read_csv(csv_file)
    dfs.append(df)

In [ ]:
df = dfs[2]

,CHARTDATE,TEXT,FIRST_OCCURANCE
0,2123-08-23,[**2123-8-23**] 12:30 PM\n CHEST (PORTABLE AP)...,0
1,2123-08-24,Smicu nsg admission note\n65 women recently dx...,0
2,2123-08-25,[**2123-8-25**] 9:02 PM\n CHEST (PORTABLE AP) ...,0
3,2123-08-26,[**2123-8-26**] 2:17 PM\n MR HEAD W & W/O CONT...,0


In [ ]:
df

,CHARTDATE,TEXT,FIRST_OCCURANCE
0,2123-08-23,[**2123-8-23**] 12:30 PM\n CHEST (PORTABLE AP)...,0
1,2123-08-24,Smicu nsg admission note\n65 women recently dx...,0
2,2123-08-25,[**2123-8-25**] 9:02 PM\n CHEST (PORTABLE AP) ...,0
3,2123-08-26,[**2123-8-26**] 2:17 PM\n MR HEAD W & W/O CONT...,0
4,2123-08-27,[**2123-8-27**] 12:51 AM\n CHEST (PORTABLE AP)...,0
5,2123-08-29,[**2123-8-29**] 2:32 PM\n CHEST (PA & LAT); CH...,0
6,2123-08-30,[**2123-8-30**] 6:18 PM\n CHEST (PORTABLE AP) ...,0
7,2123-08-31,[**2123-8-31**] 10:57 AM\n CHEST (PORTABLE AP)...,0
8,2123-09-01,[**2123-9-1**] 7:09 AM\n CHEST (PORTABLE AP) ...,0
9,2123-09-02,[**2123-9-2**] 8:33 AM\n CHEST (PORTABLE AP) ...,0


In [ ]:
import re
import os
import glob
import pandas as pd
from tqdm import tqdm

In [ ]:
!unzip /content/patients_datewise_gpt35-20231210T194303Z-001.zip

Archive:  /content/patients_datewise_gpt35-20231210T194303Z-001.zip
  inflating: patients_datewise_gpt35/gpt35_16k_fo_text_49079_100234.csv  
  inflating: patients_datewise_gpt35/gpt35_16k_fo_text_1104_113352.csv  
  inflating: patients_datewise_gpt35/gpt35_16k_fo_text_48711_196486.csv  
  inflating: patients_datewise_gpt35/gpt35_16k_fo_text_48779_193716.csv  
  inflating: patients_datewise_gpt35/gpt35_16k_fo_text_48038_141060.csv  
  inflating: patients_datewise_gpt35/gpt35_16k_fo_text_1066_154406.csv  
  inflating: patients_datewise_gpt35/gpt35_16k_fo_text_4851_130396.csv  
  inflating: patients_datewise_gpt35/gpt35_16k_fo_text_1042_135250.csv  
  inflating: patients_datewise_gpt35/gpt35_16k_fo_text_48388_156310.csv  
  inflating: patients_datewise_gpt35/gpt35_16k_fo_text_44277_123123.csv  
  inflating: patients_datewise_gpt35/gpt35_16k_fo_text_44530_159574.csv  
  inflating: patients_datewise_gpt35/gpt35_16k_fo_text_48344_122365.csv  
  inflating: patients_datewise_gpt35/gpt35_16k_f

In [ ]:
directory_path = 'Patients_Datewise_Texts_With_First_Occurance'

csv_files = glob.glob(os.path.join(directory_path, '*.csv'))
dfs = []

already_saved = glob.glob(os.path.join('patients_datewise_gpt35', '*.csv'))

for csv_file in tqdm(csv_files):
    out_filename = 'gpt35_16k_' + csv_file.split('\\')[-1]
    if os.path.join('patients_datewise_gpt35', out_filename) in already_saved:
        continue

    df = pd.read_csv(csv_file)
    res, summaries, answers = [], [], []
    summary = ''
    token_len_exceeded_error = False
    for i, row in df.iterrows():
        try:
            text = 'Prior patient reports summary: ' + summary + '\nCurrent report:\n' + row['TEXT']
            start = time.time()
            completion = client.chat.completions.create(
                model="gpt-3.5-turbo-1106",
                messages=[
                    {"role": "system", "content": "You are an assistant for a medical practitioner, skilled in analyzing and summarizing medical notes and reports."},
                    {"role": "user", "content": text + ".\n Using the above medical notes, please give your diagnosis and also determine if the patient has pulmonary embolism. Summarize the report (start with Summary:), give your short diagnosis (start with Diagnosis:), and answer the pulmonary embolism question only with a yes or no. If pulmonary embolism cannot be established, please say no and nothing more."},
                ]
            )
            gentime = time.time() - start
            ans = completion.choices[0].message.content
            pattern = re.compile(r'Summary:(.*?)(?:Pulmonary embolism:)(.*)', re.DOTALL | re.IGNORECASE)
            summary_match = pattern.search(ans)
            summary = summary_match.group(1).strip() if summary_match else summary
            summaries.append(summary)

            # pe_match = re.search(r'Pulmonary Embolism: (.*)', ans, re.IGNORECASE)
            pe_text = summary_match.group(2).strip() if summary_match else ""
            res.append(pe_text)
            answers.append(ans)
            # print(ans, '\n', gentime)
        except Exception as e:
            print(e)
            token_len_exceeded_error = True
            break

    if token_len_exceeded_error:
        continue

    try:
        assert len(res) == len(df) == len(summaries) == len(answers), 'Lengths mismatch'
    except AssertionError:
        continue

    df['Summary'] = summaries
    df['PE_Result_GPT35_16k'] = res
    df['Answers'] = answers

    df.to_csv('patients_datewise_gpt35/' + out_filename, index=False)

In [ ]:
# df.drop(columns=['Summary', 'PE_Result_GPT35_16k', 'Answers'], inplace=True)
# df.head(2)

In [ ]:
# start = time.time()
# completion = client.chat.completions.create(
#   model="gpt-3.5-turbo",
#   messages=[
#     {"role": "system", "content": "You are an assistant for a medical practitioner, skilled in analyzing and summarizing medical notes and reports."},
#     {"role": "user", "content": "f{} Using the above medical notes, please give your diagnosis and also determine if the patient has pulmonary embolism. Keep your diagnosis short, and answer the pulmonary embolism question with a yes or no."},
#   ]
# )
# gentime = time.time() - start
# print(completion.choices[0].message, '\n', gentime)

ChatCompletionMessage(content="In the realm of code, where logic pervades,\nI'll weave a tale of a looping crusade.\nA concept profound, yet elegantly clear,\nRecursion, dear friends, is what we shall hear.\n\nImagine a function, a humble charmer,\nWhose power lies not in its initial armor.\nA call, it makes, to itself with grace,\nLike mirrors reflecting, within time and space.\n\nA seed of thought, a problem to unravel,\nEach iteration, a layer to travel.\nWith each recursion, it breaks it down,\nInto smaller fragments, truth shall be found.\n\nA fractal of code, a path untold,\nExpanding dimensions, behold, behold!\nIt dances with grace, in a mystical trance,\nUnraveling the puzzle, giving it a chance.\n\nLike an echo resounding, back and forth,\nIt explores the labyrinth, south, east, and north.\nWith keen observation, it scrutinizes,\nUnraveling mysteries, solving the prizes.\n\nBut beware, my friends, for recursion can be sly,\nCaught in a loop, it may make you cry.\nAn end condi

In [ ]:
print(dfs[2]["TEXT"].iloc[2])

Inference

In [111]:
!unzip "/content/patients_datewise_gpt35-20231210T194303Z-001.zip"

Archive:  /content/patients_datewise_gpt35-20231210T194303Z-001.zip
replace patients_datewise_gpt35/gpt35_16k_fo_text_49079_100234.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: Archive:  /content/patients_datewise_gpt35_first_occurence_negation_handled.zip
   creating: content/patients_datewise_gpt35_first_occurence_negation_handled/
  inflating: content/patients_datewise_gpt35_first_occurence_negation_handled/fo_gpt35_16k_fo_text_23790_133505.csv  
  inflating: content/patients_datewise_gpt35_first_occurence_negation_handled/fo_gpt35_16k_fo_text_16264_158183.csv  
  inflating: content/patients_datewise_gpt35_first_occurence_negation_handled/fo_gpt35_16k_fo_text_4668_140989.csv  
  inflating: content/patients_datewise_gpt35_first_occurence_negation_handled/fo_gpt35_16k_fo_text_19147_159775.csv  
  inflating: content/patients_datewise_gpt35_first_occurence_negation_handled/fo_gpt35_16k_fo_text_11866_148480.csv  
  inflating: content/patients_datewise_gpt35_first_occurence_negation_handled/

In [113]:
!unzip "/content/patients_datewise_gpt35_first_occurence_negation_handled.zip"

Archive:  patients_datewise_gpt35_first_occurence_negation_handled.zip
replace content/patients_datewise_gpt35_first_occurence_negation_handled/fo_gpt35_16k_fo_text_23790_133505.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
import os
import glob
import pandas as pd
directory_path = '/content/patients_datewise_gpt35/'

csv_files = glob.glob(os.path.join(directory_path, '*.csv'))
patient_ids = []
dfs = []
for csv_file in csv_files:
    patient_ids.append(csv_file.split('_')[-2] + ' ' + csv_file.split('_')[-1].split('.')[0])
    df = pd.read_csv(csv_file)
    dfs.append(df)

In [ ]:
filtered_dfs = []
for i, df in enumerate(dfs):
    index_of_fo = (df['FIRST_OCCURANCE'] == 1).idxmax()
    # index_of_fpred1 = (df['PE_Result_GPT35_16k'] == 'Yes').idxmax()
    # index_of_fpred2 = (df['PE_Result_GPT35_16k'] == 'YES').idxmax()
    # index_of_fpred = df[df['PE_Result_GPT35_16k'].str.contains('Yes|YES', case=False)].index.min()
    index_of_fpred = df['PE_Result_GPT35_16k'].str.upper().eq('YES').idxmax() if any(df['PE_Result_GPT35_16k'].str.upper().eq('YES')) else float('inf')
    if(index_of_fo > index_of_fpred):
        # print(index_of_fpred1, index_of_fpred2)
        filtered_dfs.append((patient_ids[i], df))

In [40]:
len(filtered_dfs)

144

In [120]:
a = set([tup[0] for tup in filtered_dfs])
b = set(['20833 108482',
 '29002 162289',
 '3363 142100',
 '23291 103175',
 '31757 193966',
 '49079 100234',
 '12715 174198',
 '25325 109146',
 '28636 141318',
 '16099 143325',
 '10315 110147',
 '20209 126687',
 '2158 123815',
 '15347 134311',
 '43731 190000',
 '32094 146827',
 '17793 122607',
 '48038 141060',
 '47731 152340',
 '46093 139857',
 '29668 183460',
 '28076 144752',
 '17125 180931',
 '47118 114016',
 '26578 196923',
 '20471 158622',
 '23219 186851',
 '21514 131822',
 '19147 159775',
 '26712 194125',
 '20778 113683',
 '29692 128551',
 '1949 122008',
 '18674 178865',
 '27024 198972',
 '3181 181599',
 '14037 185837',
 '45321 197549',
 '22543 146274',
 '28315 139009',
 '22674 107142',
 '10967 184612',
 '3216 198299',
 '24615 156055',
 '30868 196214',
 '28666 151351',
 '22330 136390',
 '32013 185178',
 '41022 147181',
 '14517 163144',
 '2319 143972',
 '2151 153468',
 '12968 126751',
 '29838 164557',
 '19372 117365',
 '28623 152544',
 '43149 108346',
 '23792 127474',
 '3695 103997',
 '27576 113975',
 '12805 151145',
 '1492 129648',
 '25724 107008',
 '17602 108035',
 '16446 174937',
 '18718 162712',
 '11866 148480',
 '15624 192513',
 '47807 151364',
 '3007 128949',
 '4851 130396',
 '10207 118574',
 '20005 145132',
 '1895 100751',
 '24975 101092',
 '272 164716',
 '22080 124978',
 '48044 104324',
 '46397 130076',
 '22129 178957',
 '10483 131464',
 '10017 199207',
 '40546 158114',
 '26920 139698',
 '1630 109760',
 '27017 169236',
 '32097 175321',
 '22076 154613',
 '28869 118633',
 '10258 152188',
 '31779 179455',
 '20324 113071',
 '16966 123375',
 '3286 133404',
 '17977 198062',
 '14710 181548',
 '28034 157954',
 '40867 148920',
 '23432 131885',
 '31719 125410',
 '4734 118555',
 '3977 198367',
 '13076 178157',
 '45599 195826',
 '11762 130136',
 '16020 196065',
 '29293 199898',
 '31141 171547',
 '20568 116797',
 '31080 199082',
 '12183 180744',
 '16786 122244',
 '3941 196745',
 '11320 140887',
 '12810 164099',
 '2148 123870',
 '48388 156310',
 '24109 155286',
 '24512 188843',
 '23130 154351',
 '45291 144991',
 '47271 114608',
 '22026 141081',
 '20485 175359',
 '12803 100030',
 '11615 131143',
 '3134 198911',
 '47739 178987',
 '26658 161024',
 '10633 174576',
 '43881 167021',
 '31891 192838',
 '16985 138680',
 '27152 108589',
 '29959 167558',
 '13414 107449',
 '27890 165787',
 '21662 118736',
 '47693 188536',
 '23792 143170',
 '31692 174449',
 '29123 185864',
 '16890 172902',
 '25933 145785'])

In [122]:
len(a.difference(b))

81

In [125]:
print(len(a))
print(len(b))

144
144


In [114]:
filtered_dfs[25]

('41701 149780',
     CHARTDATE                                               TEXT  \
 0  2178-01-17  [**2178-1-17**] 11:55 AM\n BILAT LOWER EXT VEI...   
 1  2178-01-19  [**2178-1-19**] 6:19 PM\n CT ABD & PELVIS WITH...   
 2  2178-01-21  [**2178-1-21**] 12:16 PM\n IVC GRAM/FILTER    ...   
 3  2178-01-23  Admission Date:  [**2178-1-17**]              ...   
 
    FIRST_OCCURANCE                                            Summary  \
 0                0  The 68-year-old male patient presented with sw...   
 1                1  The 68-year-old male patient presented with ex...   
 2                1  The 68-year-old male patient underwent an IVC ...   
 3                1  A 68-year-old male patient with a history of e...   
 
   PE_Result_GPT35_16k                                            Answers  
 0                 Yes  Summary:\nThe 68-year-old male patient present...  
 1                 Yes  Summary:\nThe 68-year-old male patient present...  
 2                 Yes  Summary:\nTh

In [115]:
print(filtered_dfs[25][0])

41701 149780


In [118]:
print(filtered_dfs[25][1]["TEXT"].iloc[0])

[**2178-1-17**] 11:55 AM
 BILAT LOWER EXT VEINS                                           Clip # [**Clip Number (Radiology) 18252**]
 Reason: SWELLING DVT?
 ______________________________________________________________________________
 [**Hospital 2**] MEDICAL CONDITION:
  68 year old man with swelling, pain LLE, no h/o trauma
 REASON FOR THIS EXAMINATION:
  dvt?
 ______________________________________________________________________________
 WET READ: OXZa SAT [**2178-1-17**] 12:42 PM
  Extensive DVT in left lower extremity involving femoral, superficial femoral
  and popliteal with small amount of residual flow in those veins. calf veins
  not visualized and may be occluded.
 ______________________________________________________________________________
                                 FINAL REPORT
 INDICATION:  Left lower extremity swelling and pain.

 TECHNIQUE:  Ultrasound of the bilateral lower extremities.

 COMPARISON:  None available.

 FINDINGS:  Grayscale and Doppler son[**

In [ ]:
filtered_dfs[0]['PE_Result_GPT35_16k'].str.upper().eq('YES').idxmax()

11

In [ ]:
df = pd.read_csv("/content/patients_datewise_gpt35/gpt35_16k_fo_text_2639_157999.csv")

In [ ]:
df

,CHARTDATE,TEXT,FIRST_OCCURANCE,Summary,PE_Result_GPT35_16k,Answers
0,2113-09-29,Ectopic atrial rhythm. Left axis deviation. Po...,0,The current report indicates an ectopic atrial...,No.,Summary:\nThe current report indicates an ecto...
1,2113-09-30,PATIENT/TEST INFORMATION:\nIndication: Coronar...,0,The patient's echocardiogram report indicates ...,No,Summary:\nThe patient's echocardiogram report ...
2,2113-10-01,[**2113-10-1**] 9:59 AM\n CHEST (PORTABLE AP) ...,1,"The patient, a 64-year-old woman with COPD, CA...",No,"Summary:\nThe patient, a 64-year-old woman wit..."
3,2113-10-03,[**2113-10-3**] 8:11 AM\n CHEST (PORTABLE AP) ...,1,The 64-year-old female patient with a history ...,No,Summary:\nThe 64-year-old female patient with ...
4,2113-10-04,[**2113-10-4**] 5:28 AM\n CHEST (PORTABLE AP) ...,1,The 64-year-old female patient with a history ...,No.,Summary:\nThe 64-year-old female patient with ...
5,2113-10-05,[**2113-10-5**] 8:07 AM\n CHEST (PORTABLE AP) ...,1,The 65-year-old female patient presented with ...,No,Summary:\nThe 65-year-old female patient prese...
6,2113-10-06,[**2113-10-6**] 8:18 AM\n CHEST (PORTABLE AP) ...,1,The 65-year-old female patient with a history ...,No,Summary:\nThe 65-year-old female patient with ...
7,2113-10-07,[**2113-10-7**] 9:57 AM\n CHEST (PORTABLE AP) ...,1,A 65-year-old female with a history of dilated...,No,Summary:\nA 65-year-old female with a history ...
8,2113-10-08,[**2113-10-8**] 3:37 AM\n CHEST (PORTABLE AP) ...,1,A 65-year-old female with a history of dilated...,Yes,Summary:\nA 65-year-old female with a history ...
9,2113-10-09,[**2113-10-9**] 3:49 AM\n CHEST (PORTABLE AP) ...,1,A 65-year-old woman with a history of dilated ...,No,Summary:\nA 65-year-old woman with a history o...


In [ ]:
(df['FIRST_OCCURANCE'] == 1)

0     False
1     False
2      True
3      True
4      True
5      True
6      True
7      True
8      True
9      True
10     True
11     True
12     True
13     True
14     True
Name: FIRST_OCCURANCE, dtype: bool

In [ ]:
print(df["TEXT"].iloc[2])

In [ ]:
print(df["Answers"].iloc[2])

In [ ]:
print(dfs[0])

In [ ]:
patient_ids[0]